### NOTE: Neither lines of code nor text have been proofread and the Face-Recognition module does not work with Windows

In [1]:
from pathlib import Path
import urllib.request

import face_recognition   # Does not work with Windows, so Jupiter Notebook should be run in Linux
import cv2 as cv
import numpy as np
import requests
import imutils

import pickle
from collections import Counter

In [3]:
"""Captures from a mobile phone camera but does not save the frames"""

url = "http://xxxxx/shot.jpg"

while True: 
    frame_req = requests.get(url)
    frame_arr = np.array(bytearray(frame_req.content), dtype=np.uint8)
    frame = cv.imdecode(frame_arr, -1)
    frame = imutils.resize(frame, width=640, height=400)
    cv.imshow("Android_cam", frame)

    if cv.waitKey(1) == ord('q'):  # The user should open capture window
        break                      # striking the 'q' key ends the capture


error: OpenCV(4.8.1) /io/opencv/modules/imgcodecs/src/loadsave.cpp:802: error: (-215:Assertion failed) !buf.empty() in function 'imdecode_'


The above is not very good for face recognition; see
<br>
https://www.linkedin.com/advice/1/how-can-you-ensure-facial-recognition-accuracy-jr7de?utm_source=share&utm_medium=member_android&utm_campaign=share_via

In [4]:
cv.destroyAllWindows()

Face recognition from IP webcam (mobile phone camera)

In [1]:
url = "http://xxxxx/shot.jpg"


# Defining the path for encodings from training image files
DEFAULT_ENCODINGS_PATH = Path("output/encodings.pkl")

# Defining tolrance and bounding boxes for faces
TOLERANCE = 0.4
FRAME_THICKNESS = 3
FONT_THICKNESS = 2
MODEL = "hog" # cnn

encodings_location = DEFAULT_ENCODINGS_PATH

while True:  
    with encodings_location.open(mode="rb") as f:
        loaded_encodings = pickle.load(f)

    frame_req = requests.get(url)
    frame_arr = np.array(bytearray(frame_req.content), dtype=np.uint8)
    frame = cv.imdecode(frame_arr, -1)
    frame = imutils.resize(frame, width=640, height=480)
    input_face_locations = face_recognition.face_locations(
        frame, model=MODEL
    )
    input_face_encodings = face_recognition.face_encodings(
        frame, input_face_locations
    )

    for bounding_box, unknown_encoding in zip(
        input_face_locations, input_face_encodings
    ):

        boolean_matches = face_recognition.compare_faces(
            loaded_encodings["encodings"], unknown_encoding, TOLERANCE
        )

        votes = Counter(
            name
            for match, name in zip(boolean_matches, loaded_encodings["names"])
            if match
        )
        
        # Define and execute bounding box for match   
        top_left = (bounding_box[3], bounding_box[0])
        bottom_right = (bounding_box[1], bounding_box[2])
        color = [255, 0, 0]
        cv.rectangle(frame, top_left, bottom_right, color, FRAME_THICKNESS)
        
        # Insert name of match in box
        if votes:
            name = votes.most_common(1)[0][0]
            top_left = (bounding_box[3], bounding_box[2])
            bottom_right = (bounding_box[1], bounding_box[2]+22)
            cv.rectangle(frame, top_left, bottom_right, color, cv.FILLED)
            cv.putText(frame, name, (bounding_box[3]+10, bounding_box[2]+15),
                        cv.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200), FONT_THICKNESS)
          
        else:
            name = "Unknown"
            top_left = (bounding_box[3], bounding_box[2])
            bottom_right = (bounding_box[1], bounding_box[2]+22)
            cv.rectangle(frame, top_left, bottom_right, color, cv.FILLED)
            cv.putText(frame, name, (bounding_box[3]+10, bounding_box[2]+15),
                        cv.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200), FONT_THICKNESS)

        cv.imshow("Android_cam", frame)
        if cv.waitKey(1) == ord('q') or ConnectionRefusedError:
            break
    
            


In [6]:
cv.destroyAllWindows()

Face-recognition from video

In [11]:
# Defining the path for encodings from training image files
DEFAULT_ENCODINGS_PATH = Path("output/encodings.pkl")

# Defining bounding boxes for faces
FRAME_THICKNESS = 3
FONT_THICKNESS = 2
MODEL = "hog" # cnn

image_location = input('Enter video file name and its directory : ')
video = cv.VideoCapture(image_location)
width = video.get(cv.CAP_PROP_FRAME_WIDTH)   # 3
height = video.get(cv.CAP_PROP_FRAME_HEIGHT)  # 4

encodings_location = DEFAULT_ENCODINGS_PATH

# Define the codec and create VideoWriter object
codec = cv.VideoWriter_fourcc(*"mp4v")
destination = input("Enter output file name and its directory : ")
out = cv.VideoWriter(destination, codec, 20.0, (int(width), int(height)))

with encodings_location.open(mode="rb") as f:
    loaded_encodings = pickle.load(f)
    
while video.isOpened():
    ret, frame = video.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    else:            
        input_face_locations = face_recognition.face_locations(
            frame, model=MODEL
        )
        input_face_encodings = face_recognition.face_encodings(
            frame, input_face_locations
        )

        for bounding_box, unknown_encoding in zip(
            input_face_locations, input_face_encodings
        ):
            
            boolean_matches = face_recognition.compare_faces(
                loaded_encodings["encodings"], unknown_encoding
            )

            votes = Counter(
                name
                for match, name in zip(boolean_matches, loaded_encodings["names"])
                if match
            )
            
            # Define and execute bounding box for match   
            top_left = (bounding_box[3], bounding_box[0])
            bottom_right = (bounding_box[1], bounding_box[2])
            color = [255, 0, 0]
            cv.rectangle(frame, top_left, bottom_right, color, FRAME_THICKNESS)

            # Insert name of match in box
            if votes:
                name = votes.most_common(1)[0][0]
                top_left = (bounding_box[3], bounding_box[2])
                bottom_right = (bounding_box[1], bounding_box[2]+22)
                cv.rectangle(frame, top_left, bottom_right, color, cv.FILLED)
                cv.putText(frame, name, (bounding_box[3]+10, bounding_box[2]+15),
                            cv.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200), FONT_THICKNESS)

            else:
                name = "Unknown"
                top_left = (bounding_box[3], bounding_box[2])
                bottom_right = (bounding_box[1], bounding_box[2]+22)
                cv.rectangle(frame, top_left, bottom_right, color, cv.FILLED)
                cv.putText(frame, name, (bounding_box[3]+10, bounding_box[2]+15),
                            cv.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200), FONT_THICKNESS)
                
            out.write(frame)
#             cv.imshow('frame', frame)
#             if cv.waitKey(1) == ord('q'):
#                 break


Enter video file name and its directory : ../bank_face_videos/Madonna on the Arsenio Hall Show_clip.mp4
Enter output file name and its directory : ../bank_face_videos/Madonna on the Arsenio Hall Show_IDclip2.mp4
Can't receive frame (stream end?). Exiting ...


In [12]:
video.release()
out.release()

In [10]:
cv.destroyAllWindows()

In [14]:
video = cv.VideoCapture(image_location)
width = video.get(cv.CAP_PROP_FRAME_WIDTH)   # 3
height = video.get(cv.CAP_PROP_FRAME_HEIGHT)  # 4

(width, height)

(1280.0, 720.0)